In [1]:
import pandas as pd
import spacy
from concurrent.futures import ThreadPoolExecutor

# Загрузка данных из Excel
df = pd.read_excel("ner_data_test.xlsx")

In [2]:
# Описание тегов
tag_descriptions = {
    "O": "Неопределенный",
    "B-Дата": "Дата",
    "B-бренд": "Бренд",
    "B-вид спорта": "Вид спорта",
    "B-видеоигра": "Видеоигра",
    "B-команда": "Команда",
    "B-лига": "Лига",
    "B-локация": "Локация",
    "B-модель": "Модель",
    "B-название проекта": "Название проекта",
    "B-организация": "Организация",
    "B-персона": "Персона",
    "B-сезон": "Сезон",
    "B-серия": "Серия",
    "I-Дата": "Дата (продолжение)",
    "I-бренд": "Бренд (продолжение)",
    "I-вид спорта": "Вид спорта (продолжение)",
    "I-видеоигра": "Видеоигра (продолжение)",
    "I-команда": "Команда (продолжение)",
    "I-лига": "Лига (продолжение)",
    "I-локация": "Локация (продолжение)",
    "I-модель": "Модель (продолжение)",
    "I-название проекта": "Название проекта (продолжение)",
    "I-организация": "Организация (продолжение)",
    "I-персона": "Персона (продолжение)",
    "I-сезон": "Сезон (продолжение)",
    "I-серия": "Серия (продолжение)",
}

In [3]:

# Загрузка spaCy модели для русского языка
nlp = spacy.load("ru_core_news_lg")

# Функция для извлечения тегов из одной строки текста
def extract_tags(text):
    doc = nlp(text)
    tags = []
    current_label = ""
    for token in doc:
        label = tag_descriptions.get(token.ent_iob_ + "-" + token.ent_type_, "O")
        if label != current_label:
            if current_label != "O":
                tags.append(current_label)
            current_label = label
        tags.append(token.text)
    if current_label != "O":
        tags.append(current_label)
    return tags

# Функция для обработки нескольких строк данных параллельно
def process_batch(batch):
    result = []
    for text in batch:
        tags = extract_tags(text)
        result.append(tags)
    return result

# Разделение данных на пакеты для параллельной обработки
batch_size = 10
batches = [df["video_info"][i:i+batch_size] for i in range(0, len(df), batch_size)]

# Использование ThreadPoolExecutor для параллельной обработки
with ThreadPoolExecutor(max_workers=4) as executor:
    processed_batches = list(executor.map(process_batch, batches))

# Объединение результатов
tags_result = [tag for batch in processed_batches for tag in batch]

# Добавление результатов в DataFrame
df["tags"] = tags_result

# Сохранение результата в новом Excel файле
df.to_excel("output_processed_with_tags.xlsx", index=False)

# Вывод обработанных данных
print(df)

                                             video_info  entities_prediction  \
0     <НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...                  NaN   
1     <НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...                  NaN   
2     <НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...                  NaN   
3     <НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...                  NaN   
4     <НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...                  NaN   
...                                                 ...                  ...   
1601  <НАЗВАНИЕ:> Милейшие детеныши животных, которы...                  NaN   
1602  <НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...                  NaN   
1603  <НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...                  NaN   
1604  <НАЗВАНИЕ:> Александр Горелов = о вакцинации о...                  NaN   
1605  <НАЗВАНИЕ:> Смотрела УЖАСТИК и забыла, что не ...                  NaN   

                                       